In [1]:
import json
from enum import Enum
from pathlib import Path
import psycopg2
import ollama
import pandas as pd
from IPython.display import Image, Markdown, display
from tqdm import tqdm
import os
from dotenv import load_dotenv, find_dotenv
import urllib.parse
from langchain.messages import AIMessage
from langchain_ollama import ChatOllama
from langchain_deepseek import ChatDeepSeek
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_agent
from agentic_ai_tools import AccessAccountsReceivable, AccessPayments
import datetime as dt
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
from langchain_experimental.agents import create_pandas_dataframe_agent
from typing import Union, List, Dict
import datetime
from datetime import date 
from sqlalchemy import create_engine
import csv
import ast
from typing import Optional

In [2]:
# Langchain API KEY
LANGSMITH_API_KEY=os.getenv('LANGSMITH_API_KEY')
LANGSMITH_ENDPOINT=os.getenv('LANGSMITH_ENDPOINT')
# DATABASE Connection settings
DB_NAME=os.getenv('DB_NAME')
USERNAME=os.getenv('USERNAME')
PASSWORD=urllib.parse.quote(os.getenv('PASSWORD'))
HOSTNAME=os.getenv('HOSTNAME')
PORT=os.getenv('PORT')

# Creating conneciton to database

engine = create_engine(f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DB_NAME}')
#conn = psycopg2.connect(f"dbname={DB_NAME} user={USERNAME} password={PASSWORD}")
accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", engine)
payments= pd.read_sql("SELECT * FROM payments", engine)
counter = 0

In [3]:
class Context():
    """Custom runtime context schema."""
    user_id: str

In [4]:
class Invoice(BaseModel):
    invoice_number: Optional[int] = None
    date: Optional[str] = None
    customer_name: Optional[str] = None
    customer_number: Optional[int] = None
    amount: Optional[float] = None
    due_date: Optional[str] = None
    payment: Optional[float]= None
    payment_date: Optional[str] = None
    payment_id: Optional[int] = None


In [5]:
prompt ="""
You are a senior accountant responsible for accounts receivable reconciliation.

Rules:
- When reconciliation or invoice updates are requested, you determine whether the payment is related to the invoice.
- Never ask the user follow-up questions.
- Do not explain your reasoning.
- Do not return stringified data.
- Do not return different structured data.
- DO NOT nest the payments data into the updated accounts receivable information.

Output requirements:
- Return ONLY a Python dictionary.
- The dictionary must represent the UPDATED accounts receivable.
- Do not wrap the output in text, markdown, or code fences.
"""

In [6]:
# Add memory to your agent to maintain state across interactions. This allows the agent to remember previous conversations and context.
checkpointer = InMemorySaver()

In [7]:
model = ChatOllama(
    model="qwen3:8b",
    temperature=0,
) 

In [8]:
def row_gen(dataframe):
    for _, row in dataframe.iterrows():
        yield str(row.to_dict())

In [9]:
generator_ar = row_gen(accounts_receivables)


In [10]:
generator_p= row_gen(payments)

In [11]:
def update_ar(dict_input):
    if dict_input is not None:

        # Output is a string dictionary, so we will need to convert it to a regular dictonary in python
        row =  ast.literal_eval(dict_input)
        data = [row]

        with open("outputfile.csv", "w", newline='') as inp:
            column_names = list(row.keys())
            writer = csv.DictWriter(inp,fieldnames=column_names )
            writer.writeheader()
            writer.writerows(data)
            inp.close()
    else:
        pass


In [12]:
def turn_based_checker(accounts_receivable, payments, generator_ar, generator_p, counter):
    counter += 1
    column_names = list(accounts_receivable.columns)
    updated_ar = pd.DataFrame(columns=column_names)
    #print(f"Empty DF: {updated_ar}")
    for i in range(1):
        is_match = False
        row_ar = next(generator_ar)
        if not row_ar: break
        generator_p= row_gen(payments) # Recreate the payments generator, so that we do not run into a Stopiteration Error
        # Stop iteration error is raised when a generator is exhausted and will have to be created again
        for j in tqdm(range(len(payments))):
            print(f"Iteration j: {j}")
            row_p = next(generator_p)
            # Run agent
            #`thread_id` is a unique identifier for a given conversation
            config = {"configurable": {"thread_id": "{counter}"}}
            # Create the agent with default parameters
            agent = create_agent(model=model, system_prompt=prompt, checkpointer=checkpointer, response_format=Invoice)
            # Ask the agent a question
            response = agent.invoke({"messages": [{"role": "user", 
                            "content": f"""
            You have been given an accounts receivable invoice {row_ar} and a received payment {row_p}. Assess whether the payment relates to the invoice. 
            If there is no match, return an empty dictionary. If it does, UPDATE THE ACCOUNTS RECEIVABLE DATA with the relevant information from the payment. Return the updated accounts receivables data as
            a dictionary with no other additional text.
            """}]},    config=config,
                context=Context())
            
            ai_messages = [
                m for m in response["messages"]
                if isinstance(m, AIMessage)
            ]
            print(ai_messages)
            final_output = ai_messages[-2].content
            print(f"Final Output: {final_output}")
            if final_output is not None:                    
                # Convert to dictionary
                row =  ast.literal_eval(final_output)
                print(f"ROW OUTPUT:{row}")
                updated_row = pd.DataFrame(row, index=[(i+j)])
                print(f"Updated row: {updated_row}")
                is_match = True
                updated_ar = pd.concat([updated_ar, updated_row], ignore_index=True)
                #print(f"Updated AR after Match: {updated_ar}")
            else:
                pass
        
        if is_match == False:
            updated_ar = pd.concat([updated_ar, row_ar], ignore_index=True)
            #print(f"Updated AR after no match: {update_ar}")
        else:
            pass
        # Save updated DF to csv


    #print(f"Updated ar before save: {updated_ar}")
    updated_ar.to_csv('updated_ar.csv', index=False)
    #update_ar(dict_input=final_output)

            

                        

        

In [13]:
%%time
turn_based_checker(accounts_receivable=accounts_receivables, payments=payments, generator_ar=generator_ar, generator_p=generator_p, counter=counter)

  0%|          | 0/9 [00:00<?, ?it/s]

Iteration j: 0


  0%|          | 0/9 [01:22<?, ?it/s]

[AIMessage(content='{"invoice_number": 1, "date": "2025-05-01", "customer_name": "Planet Express", "customer_number": 12038, "amount": 50000.0, "due_date": "2025-05-02", "payment": 25000.0, "payment_date": "2025-01-31", "payment_id": 948347}', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2025-12-30T21:38:08.330002Z', 'done': True, 'done_reason': 'stop', 'total_duration': 68341038708, 'load_duration': 76858917, 'prompt_eval_count': 507, 'prompt_eval_duration': 1686787791, 'eval_count': 657, 'eval_duration': 65891009138, 'logprobs': None, 'model_name': 'qwen3:8b', 'model_provider': 'ollama'}, id='lc_run--76d4c2de-c03e-428e-ac87-bc29ffdaa156-0', usage_metadata={'input_tokens': 507, 'output_tokens': 657, 'total_tokens': 1164}), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2025-12-30T21:38:09.521567Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1174034875, 'load_duration': 89682834, 'prompt_eva

SyntaxError: invalid syntax (<unknown>, line 0)